In [3]:
import pandas as pd
import numpy as np
import pythainlp
import pickle
import tensorflow as tf
import re
import emoji
import matplotlib.pyplot as plt

from pythainlp.corpus.common import thai_stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout
from keras.utils import pad_sequences
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau

In [78]:
# Set the maximum number of words to keep based on word frequency
max_words = 10000
maxlen = 128  # Set the maximum sequence length
# Load the data from the CSV file
df = pd.read_csv('C:/Sentiment-Analysis-Model/datasets/datasetSUMTH.csv')
# dfpos = pd.read_csv('C:/Sentiment-Analysis-Model/datasets/datasetEN_filtered_POS.csv')
# dfneg = pd.read_csv('C:/Sentiment-Analysis-Model/datasets/datasetEN_filtered_NEG.csv')
# dfneu = pd.read_csv('C:/Sentiment-Analysis-Model/datasets/datasetEN_filtered_NEU.csv')

# df = pd.concat([df, dfpos, dfneg, dfneu], ignore_index=True)

data = df.drop_duplicates().copy()  # Make a copy of the DataFrame

# Define preprocess for Thai text
def preprocess_text(text):
    if pd.isnull(text):  # Check if text is NaN
        return ''
    # tokenize the text
    text = emoji.demojize(text)
    text = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "ๆ", "ฯ"))
    text = re.sub(r'[a-zA-Z]', '', text)  # Remove English characters
    text = " ".join(word for word in text)
    text = "".join(word for word in text.split() if word.lower() not in thai_stopwords())
    tokens = pythainlp.word_tokenize(str(text), engine='newmm')
    # join the tokens back into a single string
    text = " ".join(tokens)
    # remove non-alphabetic characters and extra whitespaces
    text = re.sub('[^A-Za-zก-๙]+', ' ', text).strip()
    return text

# Apply the preprocessing function to the 'text' column using loc
data.loc[:, 'text'] = data['text'].apply(preprocess_text)

# Split the data into text and labels
x = data['text'].values
y = data['sentiment'].values

# Filter out empty texts (NaN)
non_empty_indices = ~pd.isnull(x)
x = x[non_empty_indices]
y = y[non_empty_indices]

# Convert the sentiment labels to one-hot encoded vectors
y = pd.get_dummies(y).values

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Create the Tokenizer object without passing a custom tokenizer function
tokenizer = Tokenizer(num_words=max_words, filters='', lower=False, oov_token='<UNK>')
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

# Pad the sequences to a fixed length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

print("maxlen: ", maxlen)



maxlen:  128


In [80]:
# Create the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 128, 128)          1280000   
                                                                 
 conv1d_5 (Conv1D)           (None, 124, 128)          82048     
                                                                 
 global_max_pooling1d_5 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 3)                 195       
                                                      

In [81]:
# Train the model
filename = 'sentiment_analysis_modelTH.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.1, min_lr=0.000001)
callbacks_list = [checkpoint, learning_rate_reduction]

model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test), callbacks=callbacks_list)

Epoch 1/5
354/355 [============================>.] - ETA: 0s - loss: 0.7987 - accuracy: 0.6352
Epoch 1: val_loss improved from inf to 0.61433, saving model to sentiment_analysis_modelTH.h5
355/355 [==============================] - 16s 42ms/step - loss: 0.7979 - accuracy: 0.6356 - val_loss: 0.6143 - val_accuracy: 0.7524 - lr: 0.0010
Epoch 2/5
354/355 [============================>.] - ETA: 0s - loss: 0.5107 - accuracy: 0.8023
Epoch 2: val_loss improved from 0.61433 to 0.60042, saving model to sentiment_analysis_modelTH.h5
355/355 [==============================] - 15s 43ms/step - loss: 0.5107 - accuracy: 0.8023 - val_loss: 0.6004 - val_accuracy: 0.7566 - lr: 0.0010
Epoch 3/5
354/355 [============================>.] - ETA: 0s - loss: 0.3512 - accuracy: 0.8709
Epoch 3: val_loss did not improve from 0.60042
355/355 [==============================] - 15s 42ms/step - loss: 0.3510 - accuracy: 0.8710 - val_loss: 0.6437 - val_accuracy: 0.7508 - lr: 0.0010
Epoch 4/5
355/355 [===================

In [82]:
# Load the best model
model = load_model(filename)

# Compute validation loss and accuracy
val_loss, val_accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Predict classes for test data
y_pred = model.predict(x_test)
predicted_classes = np.argmax(y_pred, axis=1)

# Define class labels
class_labels = ['pos', 'neg', 'neu']

# Compute confusion matrix
confusion_mat = confusion_matrix(np.argmax(y_test, axis=1), predicted_classes)

# Print confusion matrix with class labels
print("Confusion Matrix:")
for i in range(len(class_labels)):
    label_row = ""
    for j in range(len(class_labels)):
        label_row += f"{class_labels[i]}: {confusion_mat[i][j]} "
    print(label_row)
    
print()

# Calculate precision, recall, and F1-score
# Get the true labels and predicted labels
y_true = np.argmax(y_test, axis=1)
y_pred = predicted_classes

# Calculate precision for each class
precision = precision_score(y_true, y_pred, average=None)

# Calculate recall for each class
recall = recall_score(y_true, y_pred, average=None)

# Calculate F1-score for each class
f1 = f1_score(y_true, y_pred, average=None)

# Print precision, recall, and F1-score for each class
for i in range(len(class_labels)):
    print(f"Class: {class_labels[i]}")
    print("Precision:", precision[i])
    print("Recall:", recall[i])
    print("F1-score:", f1[i])
    print()

Validation Loss: 0.6004208922386169
Validation Accuracy: 0.7566044330596924
178/178 [==============================] - 1s 5ms/step
Confusion Matrix:
pos: 1333 pos: 239 pos: 166 
neg: 194 neg: 1727 neg: 231 
neu: 173 neu: 379 neu: 1236 

Class: pos
Precision: 0.7841176470588235
Recall: 0.7669735327963176
F1-score: 0.7754508435136708

Class: neg
Precision: 0.7364605543710021
Recall: 0.8025092936802974
F1-score: 0.7680676006226373

Class: neu
Precision: 0.7568891610532762
Recall: 0.6912751677852349
F1-score: 0.7225957322420345



In [101]:
# Save the trained model and the tokenizer to disk
model.save('sentiment_analysis_modelTH.h5')
with open('tokenizerTH.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)

In [103]:
#load model and tokenizer
with open('tokenizerTH.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

model = load_model('sentiment_analysis_modelTH.h5')

In [104]:
new_text = 'เกมดีมากค่ะ ตัวละครเเต่ละตัว น่ารักมาก ชอบเดย่าสุด โครตเเจ่ม ดีไซน์ตัวละครโคตรเท่'
new_text = preprocess_text(new_text)
print('new_text:', new_text)
new_text = tokenizer.texts_to_sequences([new_text])  # Convert text to sequences of integers
new_text = pad_sequences(new_text, maxlen=maxlen)
# Make the prediction
prediction = model.predict(new_text)[0]

# Get the predicted sentiment and confidence level
sentiments = ['Negative', 'Neutral', 'Positive']
sentiment = sentiments[np.argmax(prediction)]
confidence = np.max(prediction)

# Display the result
print(f'{sentiment} ({confidence * 100:.2f}%)')

new_text: เกม ดีมาก ค่ะ ตัวละคร เเต่ ละ ตัว น่ารัก มาก ชอบ เด ย่า สุด โค รตเเจ่ มดี ไซน์ ตัวละคร โคตร เท่
1/1 [==============================] - 0s 118ms/step
Positive (91.28%)
